In [1]:
import os
import torch
import soundfile as sf

from transformers import pipeline
from transformers.utils import is_flash_attn_2_available
from jiwer import wer

ModuleNotFoundError: No module named 'nvidia_smi'

#### Load models with flash attention

Loading models sizes with flash attention. For every loading and inference, compute and store the memory usage, word error rate and real-time factor

In [ ]:
tiny = 'openai/whisper-tiny'
base = 'openai/whisper-base'
small = 'openai/whisper-small'
medium = 'openai/whisper-medium'
large = 'openai/whisper-large-v3'

models = [tiny, base, small, medium, large]

audio_files = os.listdir('audio_files')

task = 'automatic-speech-recognition'
dtype = torch.float16
device = 'cuda:0'
kwargs = {'attn_implementation': 'flash_attention_2'} if is_flash_attn_2_available() else {'attn_implementation': 'sdpa'}

Create a file to store test results

In [ ]:
results_file = 'whisper_results.txt'
if os.path.exists(results_file):
    os.remove(results_file)

header = "******** Whisper Automatic Speech Recognition test ********\n"
underline = "___________________________________________________________\n"

with open(results_file, 'a') as f:
    f.write(header)
    f.write(underline)

#### Testing pipeline

In [ ]:
for model in models:
    torch.cuda.empty_cache()
    asr_pipeline = pipeline(task=task,
                            model=model,
                            torch_dtype=dtype,
                            device=device,
                            model_kwargs=kwargs
    )

    # check how much memory the model is using when loaded
    gpu_mem_loaded = torch.cuda.memory_allocated(device)

    for audio_file in audio_files:
        torch.cuda.empty_cache()

        start = torch.cuda.Event(enable_timing=True)
        
        output = asr_pipeline(
            f'audio_files/{audio_file}.wav',
            chunck_length_s=30,
            batch_size=24,
            return_audio=True
        )

        end = torch.cuda.Event(enable_timing=True)
        end.record()
        torch.cuda.synchronize()
        time = start.elapsed_time(end)/1000

        gpu_mem_inf = torch.cuda.memory_allocated(device)
        delta_gpu_mem = gpu_mem_inf-gpu_mem_loaded

        # extract audio file duration and compute Real-Time Factor
        audio_info = sf.info(f'audio_files/{audio_file}.wav')
        rtf = time/audio_info.duration

        # compute Word Error Rate
        # read the reference transcription into a string
        
        with open(f'references/{audio_file[:-4]}.txt', 'r') as f:
            reference = f.read().replace('\n', ' ')

        result = {
                  'Model': model + '-flash',
                  'Audio file': audio_file,
                  'WER': wer,
                  'RTF': rtf,
                  'GPU memory while loaded [MB]': gpu_mem_loaded,
                  'GPU memory while inference [MB]': gpu_mem_inf,
                  'Delta GPU memory [MB]': delta_gpu_mem,
        }

        with open(results_file, 'a') as infile:
            for key, value in result.items():
                infile.write(f'{key:<32}: {value}\n')
            infile.write(underline)

Write results to file